In [ ]:
from pathlib import Path
import numpy as np
import logging

import pyvista as pv
pv.set_jupyter_backend("pythreejs")

from project_heart.enums import *
from project_heart.lv import LV

In [ ]:
directory = Path("C:/Users/igorp/OneDrive - University of South Florida/Igor/Documents/Paper 1/stress_strain_plots/ref_files")
filename = "57.32_40.94_10.08.2022_LVRNN_IDEAL_V4"
# filename = "62.71_68.65_10.08.2022_LVRNN_TYPEA_V4"

In [ ]:
from project_heart.examples import get_lv_ideal, get_lv_typeA
lv = get_lv_ideal(str(directory/filename) + ".xplt", statesfile=None)
# lv = get_lv_typeA(str(directory/filename) + ".xplt", statesfile=None)

In [ ]:
import logging

names = ["subapex", "apex", "superapex", "submid", "mid", "supermid", "subbase", "base", "superbase"]
for i, a in enumerate(np.linspace(0.1, 0.90, len(names), endpoint=True)):
    
    _ = lv.create_speckles(
        collection="circ-6",
        group="wall",
        name=names[i],
        from_nodeset=None, # use all spks
        d=2.25,
        k=a,
        normal_to=[0.0, 0.0, 1.0],
        n_subsets=6,
        subsets_criteria="angles2",
        # subsets_vector_ref=[1.0,0.0,0.0],
        n_clusters=5,
        cluster_criteria="radius2",
        t=0.0,
        log_level=logging.WARN,
    )

In [ ]:
spks = lv.get_speckles(spk_collection="circ-6", spk_group="wall")
subsets_names = ["endo", "endo-mid", "mid", "mid-epi", "epi"]
lv.split_spks_by_clusters(spks, rename_subsets=subsets_names,switch_subset_group=True)
spks_split = lv.get_speckles(spk_collection="circ-6_split")

In [ ]:
spks_mid = lv.get_speckles(spk_collection="circ-6_split", spk_name="mid", spk_group="mid-epi")
lv.plot_speckles(spks_mid, show_clusters=False)

In [ ]:
spks_base = lv.get_speckles(spk_collection="circ-6_split", spk_name="base", spk_group="endo")
lv.plot_speckles(spks_base, show_clusters=False)

In [ ]:
lv.compute_stress(spks_split, reduce_by=["group_name"], principal=False, use_axis=None, cylindrical=True, effective=False, log_level=logging.WARN)

In [ ]:
lv.compute_strain(spks_split, reduce_by=["group_name"],  principal=False, use_axis=None, cylindrical=True, effective=False, log_level=logging.WARN)

In [ ]:
_, stress_data = lv.get_metric_as_df("cylindrical_stress")
_, strain_data = lv.get_metric_as_df("cylindrical_strain")

In [ ]:
stress_df = stress_data["group_name"]
stress_df = stress_df[stress_df["timesteps"] > 0.1] - 0.1
stress_df.to_csv(str(directory/filename) + "_cylindrical_stress.csv")

In [ ]:
strain_df = strain_data["group_name"]
strain_df = strain_df[strain_df["timesteps"] > 0.1] - 0.1
strain_df.to_csv(str(directory/filename) + "_cylindrical_strain.csv")

In [ ]:
# cy_stress = lv.states.get("cylindrical_stress")
# cy_strain = lv.states.get("cylindrical_strain")

In [ ]:
# cy_stress = lv.convert_to_cylindrical_coordinates(lv.STATES.STRESS)
# cy_strain = lv.convert_to_cylindrical_coordinates(lv.STATES.LAGRANGE_STRAIN)

In [ ]:
# idx = np.arange(1,cy_stress.shape[1]+1).reshape(-1,1)

In [ ]:
# np.savetxt(
#     str(directory/filename) + "_cylindrical_stress_xplt_format.csv", 
#     np.hstack((idx, np.hstack(cy_stress))), 
#     delimiter=",", fmt="%.3f"
#     )

In [ ]:
# np.savetxt(
#     str(directory/filename) + "_cylindrical_strain_xplt_format.csv", 
#     np.hstack((idx, np.hstack(cy_strain))), 
#     delimiter=",", fmt="%.3f"
#     )